In [1]:
from requests import get
import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import ipywidgets as widgets
import math

In [6]:
def getQuery(query):
    return(get(query, timeout=20).json()['data'])

def buildQuery(api_address, filters):
    assert (len(filters) != 0)
    F = {'areaType': None,
         'areaName': None,
         'date': None,
         'newCases': None}
    F.update(filters)
    filter_string = ""
    structure_string = '"date": "date",'
    for f, v in F.items():
        if v is not None:
            filter_string += '%s=%s;' % (f, v)
        else:
            if f == 'newCases' and F['areaType'] in ['utla', 'ltla', 'region']:
                structure_string += '"newCases": "newCasesBySpecimenDate",'
            elif f == "newCases":
                structure_string += '"newCases": "newCasesByPublishDate",'
            else:
                structure_string += '"%s": "%s",' % (f, f)
    query = "%sfilters=%s&structure={%s}" % (api_address,
                                            filter_string[:-1],
                                            structure_string[:-1])
    return (query)

def cleanData(results):
    dates = []
    counts = []
    for r in results:
        date = r['date']
        d = datetime.datetime.strptime(date, '%Y-%m-%d')
        dates.append(d)
        counts.append(r['newCases'])

    sw3_counts = [np.median(counts[x:x+3]) for x in np.arange(len(counts))]
    sw7_counts = [np.median(counts[x:x+7]) for x in np.arange(len(counts))]
    return (dates, counts, sw3_counts, sw7_counts)


def plotData(clean, area_name, area_type):
    dates, counts, sw3_counts, sw7_counts = clean
    f = plt.figure(figsize=(20, 5))
    a = f.add_subplot(111)
    a.bar(dates, counts, color='darkblue')
    if area_type in ['utla', 'ltla', 'region']:
        t ='Specimen'
    else:
        t = "Published"
    a.set_title("Positive Tests by %s Date, %s" % (t, area_name), fontsize=18)
    a.plot(dates, sw3_counts, color='#fe7541B3', lw=6)
    a.plot(dates, sw7_counts, color='#41fe47B3', lw=6)
    inter = math.ceil(len(dates) / 50)
    a.set_xticks([dates[x] for x in np.arange(0, len(dates), inter)])
    a.set_xticklabels([dates[x].date() for x in np.arange(0, len(dates), inter)], rotation='vertical', fontsize=10)
    a.set_xlim(min(dates), max(dates))
    a.set_ylim(0, max(counts))
    sns.despine()
    handles = [matplotlib.patches.Patch(color='darkblue'),
               matplotlib.patches.Patch(color='#fe7541B3'),
               matplotlib.patches.Patch(color='#41fe47B3')]
    f.legend(handles=handles, labels=['Daily', '3 day median', '7 day median'], fontsize=18)
    f.tight_layout(pad=1.3)
    return (f)

def runAll(area_type, area_name):
    query = buildQuery(api_address, filters={'areaType': area_type, 'areaName': area_name})
    results = getQuery(query)
    clean = cleanData(results)
    f = plotData(clean, area_name, area_type)

def getList(typ):
    datenow = datetime.date.strftime(datetime.date.today(), "%Y-%m-%d")
    query = buildQuery(api_address, {'areaType': typ, 'date': datenow})
    results = getQuery(query)
    return ([x['areaName'] for x in results])

def changeArea(val):
    selected_area.options = getList(val['new'])

In [7]:
area_types = {'District': 'ltla',
              'County': 'utla',
              'Region': 'region',
              'Country': 'nation',
              'Total':'overview'}
api_address = 'https://api.coronavirus.data.gov.uk/v1/data?'

In [8]:
areas = widgets.Dropdown(options=area_types, description="Area Type")
area_options = getList('ltla')
selected_area = widgets.Dropdown(options=area_options, description='Area')
areas.observe(changeArea, 'value')
widgets.interactive(runAll, area_name=selected_area, area_type=areas)

interactive(children=(Dropdown(description='Area Type', options={'District': 'ltla', 'County': 'utla', 'Region…